# 短结构

In [17]:
import redis
import binascii
conn = redis.Redis()

Redis提供了一组配置选项，可以让Redis以更节约空间的方式存储较短的结构  
## 压缩列表：
* Redis在底层用双链表表示列表，用散列表示散列，用散列表+跳跃表表示有序集合
* 压缩列表会以序列化的方式存储数据，所以每次操作数据时，需要对数据进行解码操作
* 以列表为例：压缩列表的额外开销是 2字节，普通列表的额外开销是 21字节

### 配置选项
现在的配置已经不是这些了，而且还加入了 quicklist 这个数据结构，所以即使超过ziplist的限制，还是可以用quicklist包裹
* list-max-ziplist-entries 512: 允许包含的最大元素数量
* list-max-ziplist-value 64: 压缩后每个节点最大字节体积，主要是限制了value的大小
* hash-max-ziplist-entries 512
* hash-max-ziplist-value 64
* zset-max-ziplist-entries 512
* zset-max-ziplist-value 64

In [6]:
# 通过添加元素和检查表示等手段，调试一个压缩列表表示的列表对象
conn.rpush('test', 'a', 'b', 'c', 'd')

13

In [7]:
# encoding 表示编码方式
# quicklist - A doubly linked list of ziplists 一个新的数据结构，比ziplist的好处是可以仅仅修改ziplist的部分，以后更新不需要全部更新
# 相应的字节占用比ziplist多了一点
conn.debug_object('test')

{'type': 'Value',
 'at': '0x7fb2aae26490',
 'refcount': 1,
 'encoding': 'quicklist',
 'serializedlength': 55,
 'lru': 14834381,
 'lru_seconds_idle': 1,
 'ql_nodes': '1',
 'ql_avg_node': '13.00',
 'ql_ziplist_max': '-2',
 'ql_compressed': '0',
 'ql_uncompressed_size': '115'}

In [8]:
# 继续推入
conn.rpush('test', 'e', 'f', 'g', 'h')
conn.debug_object('test')

{'type': 'Value',
 'at': '0x7fb2aae26490',
 'refcount': 1,
 'encoding': 'quicklist',
 'serializedlength': 55,
 'lru': 14834383,
 'lru_seconds_idle': 0,
 'ql_nodes': '1',
 'ql_avg_node': '17.00',
 'ql_ziplist_max': '-2',
 'ql_compressed': '0',
 'ql_uncompressed_size': '127'}

In [9]:
# 推入一个超长的，导致节点存储大于限制的字符串，还是一个quicklist
conn.rpush('test', 65*'a')
conn.debug_object('test')

{'type': 'Value',
 'at': '0x7fb2aae26490',
 'refcount': 1,
 'encoding': 'quicklist',
 'serializedlength': 55,
 'lru': 14834384,
 'lru_seconds_idle': 0,
 'ql_nodes': '1',
 'ql_avg_node': '18.00',
 'ql_ziplist_max': '-2',
 'ql_compressed': '0',
 'ql_uncompressed_size': '195'}

## 集合的整数集合编码
和前面的内容不一样，集合有另外的编码方式
具体的编码方式要看存储的是什么数据
### 整数集合
* 当每个value都可以用十进制整数，且在平台有符号整数范围内+数量足够少: 按照有序整数数组的方式存储集合，叫做整数集合
* 配置：set-max-intset-entries 512: 整数的个数

In [10]:
conn.sadd('set-object', *range(500))

500

In [11]:
# 可以看到enconding 是intset
conn.debug_object('set-object')

{'type': 'Value',
 'at': '0x7fb2ac900040',
 'refcount': 1,
 'encoding': 'intset',
 'serializedlength': 1010,
 'lru': 14834822,
 'lru_seconds_idle': 15}

### 散列表
当超过限制大小是，会变成散列

In [13]:
conn.sadd('set-object', *range(500, 1000))
conn.debug_object('set-object')

{'type': 'Value',
 'at': '0x7fb2ac900040',
 'refcount': 1,
 'encoding': 'hashtable',
 'serializedlength': 2874,
 'lru': 14834995,
 'lru_seconds_idle': 0}

## 长压缩列表和大整数集合带来的性能问题
反序列化和序列化需要时间带来的性能问题  
因为版本的更新，底层数据结构已经和以前不太一样，因此测试方面的代码就不写了

### 小结
减少列表、集合、散列和有序集合的体积，可以减少内存占用  
其实缩短键名也是一种有效的减少内存的方法

# 分片技术

基于某些简单的规则将数据划分为更小空间的部分，然后根据数据所属的部分来决定将数据发送到哪个位置  
其实相当于一种新的数据结构，对外展示的功能和方法都是一样的
* 列表的分片不通过Lua脚本有点难实现，暂时不讲
* 有序集合的分片基本没有意义，也不介绍了

## 对散列进行分片

根据散列的键，确定将散列放到哪个分片中，不论是字符串还是数字，都可以这样做

In [22]:
def shard_key(base, key, total_elements, shard_size):  
    # 如果值是一个整数或者一个看上去像是整数的字符串，
    # 那么它将被直接用于计算分片 ID 。
    if isinstance(key, int) or key.isdigit():  
        # 整数键将被程序假定为连续指派的 ID ，
        # 并基于这个整数 ID 的二进制位的高位来选择分片 ID 。
        # 此外，程序在进行整数转换的时候还使用了显式的基数（以及 str()`` 函数），
        # 使得键 010 可以被转换为 10 ，而不是 8 。
        shard_id = int(str(key), 10) // shard_size     
    else:
        # 对于不是整数的键，
        # 程序将基于预计的元素总数量以及请求的分片数量，
        # 计算出实际所需的分片总数量。
        shards = 2 * total_elements // shard_size      
        # 在得知了分片的数量之后，
        # 程序就可以通过计算键的散列值与分片数量之间的模数来得到分片 ID 。
        # crc32 校验和
        shard_id = binascii.crc32(key.encode('utf-8')) % shards         
    # 最后，程序会把基础键和分片 ID 组合在一起，得出分片键。
    return "%s:%s"%(base, shard_id)  

total_elements 和 shard_size 定下了以后就不要重新赋值了，不然需要重新分片

In [23]:
def shard_hset(conn, base, key, value, total_elements, shard_size):
    # 计算出应该由哪个分片来储存值。
    shard = shard_key(base, key, total_elements, shard_size)   
    # 将值储存到分片里面。
    return conn.hset(shard, key, value)                        

def shard_hget(conn, base, key, total_elements, shard_size):
    # 计算出值可能被储存到了哪个分片里面。
    shard = shard_key(base, key, total_elements, shard_size)    
    # 取得储存在分片里面的值。
    return conn.hget(shard, key)

因为每个分片都可以用短结构进行存储了，所以总体内存会减少很多。  
直接存储字符串与存储分片散列相比，应该是分片散列效率高，占用内存更少

## 对集合进行分片
实现唯一访客计数器
* 利用uuid的前15个16进制的数来作为访客唯一键(因为误差可空，并且可以转化成10进制用整数进行分片，同时可以下降很多内存)

In [24]:
# 分片的函数可以被重复使用
def shard_sadd(conn, base, member, total_elements, shard_size):
    shard = shard_key(base,
        # 计算成员应该被储存到哪个分片集合里面；
        # 因为成员并非连续 ID ，所以程序在计算成员所属的分片之前，会先将成员转换为字符串。
        'x'+str(member), total_elements, shard_size)           
    # 将成员储存到分片里面。
    return conn.sadd(shard, member) 

In [ ]:
# 负责记录每天唯一访客人数的函数
# 在sadd之前incr就行
SHARD_SIZE = 512                       

def count_visit(conn, session_id):
    # 取得当天的日期，并生成d唯一访客计数器的键。
    today = date.today()                               
    key = 'unique:%s'%today.isoformat()                
    # 计算或者获取当天的预计唯一访客人数。
    # 因为每天的预计访客人数是可能不一样的，所以
    expected = get_expected(conn, key, today)          
 
    # 根据 128 位的 UUID ，计算出一个 56 位的 ID 。
    id = int(session_id.replace('-', '')[:15], 16)     
    # 将 ID 添加到分片集合里面。
    if shard_sadd(conn, key, id, expected, SHARD_SIZE): 
        # 如果 ID 在分片集合里面并不存在，那么对唯一访客计数器执行加一操作。
        conn.incr(key)   

# 打包存储二进制位和字节

略过